# Домашнее задание №5

https://habr.com/ru/company/mailru/blog/417767/

Урок 5. Сверточные нейронные сети для анализа текста.

Увеличить длину словаря, удалить из него мусор (rt), применить tfidf, вместо embedding использовать wordtovec, split менять на токенайзер, лемматизация, размер окна, размер словаря, заменить w2v на fasttext, сеть с различными эмбэддингами: предобученный эмбэддинг и дообучить с заморозкой весов

## Загрузка данных и библиотек

In [1]:
!pip install stop_words
!pip install pymorphy2 

  Created wheel for stop-words: filename=stop_words-2018.7.23-cp36-none-any.whl size=32916 sha256=994dafa5b989445499e1803f53c9163cacb8c09ebce3ad7414e8dcd1c81c8d24
  Stored in directory: /root/.cache/pip/wheels/75/37/6a/2b295e03bd07290f0da95c3adb9a74ba95fbc333aa8b0c7c78
Successfully built stop-words
     |████████████████████████████████| 61kB 4.7MB/s 
     |████████████████████████████████| 8.2MB 13.8MB/s 


In [35]:
from sklearn import model_selection, preprocessing, metrics
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
import re
import pandas, xgboost, numpy, textblob, string
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping
from keras import optimizers
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from string import punctuation
from gensim.models import Word2Vec
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# Устанавливаем гиперпараметры
max_words = 500
max_len = 80
num_classes = 1
epochs = 20
batch_size = 512
print_batch_n = 100

## Предобработка данных

In [4]:
data = pd.read_excel('summer.xls')

In [5]:
exclude = set(punctuation)
stop_words = set(get_stop_words("ru"))
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in stop_words]
    return " ".join(txt)

# Делаем предобработку и сокращаем количество классов до 2
data['text'] = data['Content'].apply(preprocess_text)
data = data[data['Rating'] != 3]
data['target'] = data['Rating'] > 3
data['target'] = data['target'].astype(int)
data.head()

,Rating,Content,Date,text,target
0,5,It just works!,2017-08-14,it just works,1
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,целое удобноной приложениеиз минус хотеть боль...,1
2,5,Отлично все,2017-08-14,отлично,1
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,зависать 1 работа антивирус ранее пользоваться...,1
4,5,"Очень удобно, работает быстро.",2017-08-14,удобно работать быстро,1


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['target'], test_size=0.2,
                                                    random_state=13, stratify=data['target'])

## Токенизация и фильтрация токенов по популярности

In [7]:
# Выполняем токенизацию и оставляем токены, состоящие из букв и цифр
train_corpus = " ".join(X_train)
train_corpus = train_corpus.lower()
tokens = word_tokenize(train_corpus)
tokens_filtered = [word for word in tokens if word.isalnum()]

In [8]:
# Оставляем самые часто встречающиеся токены
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'отличный',
 'хороший',
 'телефон',
 'супер']

In [9]:
len(tokens_filtered_top)

499

## Векторизация данных

In [10]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [11]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [12]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in X_train], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in X_test], dtype=np.int32)

In [13]:
x_train.shape

(15798, 80)

## Обучение сети

In [14]:
num_classes = 2
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [15]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [16]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=[keras.metrics.AUC()])

In [17]:
early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/20
28/28 [==============================] - 1s 29ms/step - loss: 0.4472 - auc: 0.8812 - val_loss: 0.3323 - val_auc: 0.9543
Epoch 2/20
28/28 [==============================] - 0s 15ms/step - loss: 0.2540 - auc: 0.9632 - val_loss: 0.2139 - val_auc: 0.9736
Epoch 3/20
28/28 [==============================] - 0s 16ms/step - loss: 0.1832 - auc: 0.9799 - val_loss: 0.1961 - val_auc: 0.9767
Epoch 4/20
28/28 [==============================] - 0s 15ms/step - loss: 0.1662 - auc: 0.9830 - val_loss: 0.1964 - val_auc: 0.9772


In [18]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

8/8 [==============================] - 0s 7ms/step - loss: 0.1787 - auc: 0.9809


Test score: 0.17871101200580597
Test accuracy: 0.980949342250824


## Сеть с предобученным W2V

In [19]:
data['text_tekens'] = data['text'].apply(word_tokenize)

In [20]:
data.head(2)

,Rating,Content,Date,text,target,text_tekens
0,5,It just works!,2017-08-14,it just works,1,"[it, just, works]"
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,целое удобноной приложениеиз минус хотеть боль...,1,"[целое, удобноной, приложениеиз, минус, хотеть..."


In [21]:
w2v_model = Word2Vec(data['text_tekens'], size=128, window=5, min_count=1, workers=8)
w2v_model.train(data['text_tekens'], total_examples=data['text_tekens'].shape[0], epochs=20)

(1508613, 1864340)

In [22]:
w2v_model.wv.vocab.keys()

dict_keys(['it', 'just', 'works', 'целое', 'удобноной', 'приложениеиз', 'минус', 'хотеть', 'большой', 'доступ', 'персональный', 'данные', 'телефонеприходиться', 'пользоваться', 'ограниченный', 'режим', 'отлично', 'зависать', '1', 'работа', 'антивирус', 'ранее', 'нормально', 'удобно', 'работать', 'быстро', 'норма', '👍👍👍', 'удобный', 'приложение', 'устраивать', 'чётко', 'отличие', 'банкомат', 'вечно', 'тупить', 'хорошо👍', 'около', 'уведомлениенельзя', 'удалять', 'стартовать', 'gps', 'sms', 'звонок', 'адресный', 'книга', 'филиал', 'фсб', 'один', 'слово', 'замечательный', 'подвисанийнуть', 'штука', 'отличный', 'практично', 'оплата', 'платёж', 'перевод', 'транспортный', 'карта', 'пополнять', 'вообще', 'жаль', 'история', 'движение', 'семья', 'замок', 'хороший', 'лишний', 'хлопоты', 'полезный', 'запуститься', 'сбербанк', 'живой', 'швейцарский', 'часы', 'супер', 'быстрокомфортный', 'поиск', 'наименование', 'служба', 'молодец', 'круто', 'хорошечный', 'пользуюсьня', 'взгляд', 'иня', 'вызывать', 

In [23]:
w2v_model['целое']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 0.42101404, -0.4924719 ,  0.2385558 , -0.03783894,  0.47519818,
        0.13250117, -0.41595206,  0.84212923,  0.15424572,  0.7015893 ,
        0.07114743, -0.8081681 ,  0.13665417,  0.26267105,  0.5685357 ,
       -0.01670249, -0.32814524,  0.10759548, -0.49051723,  0.37114137,
       -0.46228158, -0.37114105,  0.30963334,  0.32472676, -0.19132148,
       -0.35209545, -0.02051717,  0.5029785 , -0.8444485 ,  0.47622567,
        0.4452181 , -0.9050201 ,  0.03450309,  0.2546678 , -0.14276358,
        0.21068113,  0.19634207,  0.29143855, -1.1398185 , -0.02894064,
        0.02245685, -0.66479874,  0.30243674, -0.05770954, -0.790755  ,
       -0.18620342, -0.28433204, -0.49392918,  0.1296617 , -0.17120424,
        0.30883783, -0.33512932,  0.21276174, -0.0315779 ,  0.01953917,
        0.43857476, -0.62574196,  0.03348258, -0.7014211 ,  0.1512447 ,
        0.56097895,  0.12684785,  0.35872746,  0.07895179, -0.23019314,
       -0.2580232 ,  1.5590361 , -0.60535306,  0.7757542 , -0.21

In [24]:
DIM = w2v_model.vector_size
NUM = len(tokens_filtered_top) + 1
# Инициализируем матрицу embedding слоя нулями
embedding_matrix = np.zeros((NUM, DIM))
# Добавляем NUM наиболее часто встречающихся слов из обучающей выборки в embedding слой
for item in vocabulary.items():
    if item[1] >= NUM:
        break
    if item[0] in w2v_model.wv.vocab.keys():
        embedding_matrix[item[1]] = w2v_model.wv[item[0]]

In [25]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.65449744, -0.29294768,  0.29313117, ...,  0.16837452,
         0.43687716, -0.04456432],
       [ 0.92141622,  0.42437389, -0.04178007, ...,  0.61973083,
         0.48280564,  0.33568218],
       ...,
       [ 0.15808432,  0.20988321,  0.49441391, ..., -0.05211765,
         0.93426114, -0.38792726],
       [ 0.11447642,  0.20045902,  0.33656842, ...,  0.05602526,
         0.76355976, -0.30682114],
       [-0.07653184,  0.25668311,  0.18088979, ..., -0.22648831,
         0.93990356, -0.43348393]])

In [26]:
embedding_matrix.shape

(500, 128)

In [27]:
model = Sequential()
# Сначала замораживаем веса слоя
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len, weights=[embedding_matrix], trainable=False))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [28]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=[keras.metrics.AUC()])

In [29]:
early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/20
28/28 [==============================] - 0s 18ms/step - loss: 0.3179 - auc_1: 0.9428 - val_loss: 0.2605 - val_auc_1: 0.9617
Epoch 2/20
28/28 [==============================] - 0s 10ms/step - loss: 0.2422 - auc_1: 0.9656 - val_loss: 0.2366 - val_auc_1: 0.9681
Epoch 3/20
28/28 [==============================] - 0s 10ms/step - loss: 0.2219 - auc_1: 0.9709 - val_loss: 0.2246 - val_auc_1: 0.9702
Epoch 4/20
28/28 [==============================] - 0s 10ms/step - loss: 0.2082 - auc_1: 0.9747 - val_loss: 0.2112 - val_auc_1: 0.9749
Epoch 5/20
28/28 [==============================] - 0s 10ms/step - loss: 0.1924 - auc_1: 0.9783 - val_loss: 0.2032 - val_auc_1: 0.9756
Epoch 6/20
28/28 [==============================] - 0s 10ms/step - loss: 0.1811 - auc_1: 0.9805 - val_loss: 0.1981 - val_auc_1: 0.9757
Epoch 7/20
28/28 [==============================] - 0s 10ms/step - loss: 0.1724 - auc_1: 0.9822 - val_loss: 0.1964 - val_auc_1: 0.9758
Epoch 8/20
28/28 [==============================] - 0s 

In [30]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

8/8 [==============================] - 0s 7ms/step - loss: 0.1956 - auc_1: 0.9770


Test score: 0.19562692940235138
Test accuracy: 0.9770058989524841


In [42]:
model.layers[1].trainable = True
adam = optimizers.Adam(lr=0.0001)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=[keras.metrics.AUC()])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 80, 128)           64000     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 78, 128)           49280     
_________________________________________________________________
activation_3 (Activation)    (None, 78, 128)           0         
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
_________________________________________________________________
activation_4 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                

In [43]:
early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/20
28/28 [==============================] - 1s 18ms/step - loss: 0.1249 - auc_5: 0.9905 - val_loss: 0.2000 - val_auc_5: 0.9764
Epoch 2/20
28/28 [==============================] - 0s 10ms/step - loss: 0.1235 - auc_5: 0.9908 - val_loss: 0.2023 - val_auc_5: 0.9758


In [44]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

8/8 [==============================] - 0s 7ms/step - loss: 0.1979 - auc_5: 0.9771


Test score: 0.1979461908340454
Test accuracy: 0.9771061539649963


Результат с W2V и без него сопоставим и превышает заданный порог по ROC AUC, равный 0,95